In [1]:
import pandas as pd
import glob



In [2]:
meta = glob.glob( './scaffolds/CONTIGS-SCAFFOLDS-Sheet1-Nov2018/*fasta.csv')
geno = glob.glob( './orfscan/*fasta.csv')
complete_geno = glob.glob('./proteomes/done/*.csv')

header = 'targetname accession queryid accession2    E-value  score  bias   domain-E-value  domain-score  domain-bias   exp reg clu  ov env dom rep inc description'.split()

print(len(header))
print(meta)



19
['./scaffolds/CONTIGS-SCAFFOLDS-Sheet1-Nov2018/3300014208-Ga0172379-10001592-871560-40.txt.fmt1.fasta.orfs.fasta.csv', './scaffolds/CONTIGS-SCAFFOLDS-Sheet1-Nov2018/AntAceMinimDraft_10_1070366.scaffolds.fasta_scaffold19757_2-contig-3370.txt.fmt1.fasta.orfs.fasta.csv', './scaffolds/CONTIGS-SCAFFOLDS-Sheet1-Nov2018/mc1328965482-998585003-gb-LFCJ01001378.1_8-1661.txt.fmt1.fasta.orfs.fasta.csv', './scaffolds/CONTIGS-SCAFFOLDS-Sheet1-Nov2018/AntAceMinimDraft_4_1070372.scaffolds.fasta_scaffold00557_40.txt.fmt1.fasta.orfs.fasta.csv', './scaffolds/CONTIGS-SCAFFOLDS-Sheet1-Nov2018/AntAceMinimDraft_10_1070366.scaffolds.fasta_scaffold06212_4.txt.fmt1.fasta.orfs.fasta.csv', './scaffolds/CONTIGS-SCAFFOLDS-Sheet1-Nov2018/AntAceMinimDraft_18_1070375.scaffolds.fasta_scaffold13455_2.txt.fmt1.fasta.orfs.fasta.csv', './scaffolds/CONTIGS-SCAFFOLDS-Sheet1-Nov2018/3300000868-JGI12330J12834-1000008-299010-8_.txt.fmt1.fasta.orfs.fasta.csv', './scaffolds/CONTIGS-SCAFFOLDS-Sheet1-Nov2018/AntAceMinimDraft_10_

In [3]:
dfs = []
for filename in meta + geno:
    framedict = {}
    with open(filename , 'r') as infile:
        for i,line in enumerate(infile):
            if line[0] != '#':
                words = line.split()
                words[18] = ''.join([ w + ' ' for w in words[18:-1]])
                #print(words)
                framedict[i] = dict(zip( header , words[0:19]))
        df = pd.DataFrame.from_dict(framedict , orient='index')
        df.to_csv(filename + '.reformat.csv')
        dfs.append(df)
globaldf = pd.concat(dfs)
globaldf['domain'] = globaldf.accession.map(lambda x : str(int(x.split('.')[0].replace('PF',''))) )
print(globaldf)

                                           description dom domain-bias env  \
3      SMODS and SLOG-associating 2TM effector domain    1         0.0   1   
4               Concanavalin A-like lectin/glucanases    3         2.0   3   
5                                   Domain of unknown    4         0.0   4   
6                                   Bacterial Ig-like    4         0.6   4   
7                          Domain of unknown function    2         0.0   2   
8                                                  ML    1         0.0   1   
9                                    Cobalt transport    1         9.8   1   
10                         Predicted membrane protein    1         9.9   1   
11             Putative F0F1-ATPase subunit Ca2+/Mg2+    2         1.4   2   
12                        Protein of unknown function    2         0.3   2   
13                                                HYR    1         2.4   1   
14               Phosphoinositide-interacting protein    2      

In [4]:
mapdict={}
with open('rdb_pfam_to_interpro.dat','r') as mapper:
    for line in mapper:
        words=line.split()
        mapdict[words[0]] = {'interpro':words[1]}
pfam2interpor=pd.DataFrame.from_dict(mapdict, orient='index')
godict={}
with open('rdb_interpro_and_go.dat', 'r' )as godat:
    for line in godat:
        if 'IPR' in line:
            dom = line.split()[1]
        
        if 'GO:' in line and 'IPR' in dom:
            if dom not in godict:
                godict[dom]= []        
            godict[dom] =  {'goterms':''.join([ l.replace('\\','') +',' for l in line.split() if 'GO' in l ])[0:-1]}
pfam2interpor['pfam'] = pfam2interpor.index.map( lambda x: str(x))
            
ip2go = pd.DataFrame.from_dict(godict, orient='index')
def formatId(x):
    try :
        y = str(int(x.replace('IPR','')))
    except:
        y = None
    return y

ip2go['ipnum']= ip2go.index.map(formatId)
ip2go = ip2go.dropna(axis=0)
print(ip2go)

pfam2interpor = pfam2interpor.merge( ip2go, left_on= 'interpro' , right_on= 'ipnum' , how='left')
pfam2interpor = pfam2interpor.dropna(axis = 0)

print(pfam2interpor)

                                    goterms ipnum
IPR000003  GO:0005496,GO:0005634,GO:0006355     3
IPR000005  GO:0003700,GO:0005622,GO:0006355     5
IPR000006                        GO:0005505     6
IPR000009  GO:0008601,GO:0000159,GO:0007165     9
IPR000011             GO:0004839,GO:0006512    11
IPR000012                        GO:0005554    12
IPR000013  GO:0008237,GO:0005576,GO:0006508    13
IPR000015  GO:0005215,GO:0016020,GO:0006810    15
IPR000018  GO:0045028,GO:0016021,GO:0007186    18
IPR000020                        GO:0005576    20
IPR000022             GO:0004075,GO:0009343    22
IPR000023                        GO:0006096    23
IPR000024  GO:0004888,GO:0016020,GO:0007275    24
IPR000025  GO:0008502,GO:0016021,GO:0007186    25
IPR000026                        GO:0004521    26
IPR000028             GO:0004601,GO:0006804    28
IPR000031  GO:0004638,GO:0009320,GO:0006189    31
IPR000033                        GO:0016020    33
IPR000035             GO:0003905,GO:0006281    35


In [5]:
globaldf1 = globaldf.merge( pfam2interpor, left_on= 'domain' , right_on= 'pfam' , how='left')

annotdf = globaldf1[ globaldf1.goterms.notna() ]

print(annotdf.queryid.unique())

['PlaIllAssembly_1097288.scaffolds.fasta_scaffold00281_9_4'
 'PlaIllAssembly_1097288.scaffolds.fasta_scaffold00281_9_6'
 'AntAceMinimDraft_10_1070366.scaffolds.fasta_scaffold03294_14_5'
 'AntAceMinimDraft_10_1070366.scaffolds.fasta_scaffold03294_14_6'
 'AntAceMinimDraft_18_1070375.scaffolds.fasta_scaffold02754_4_3'
 'AntAceMinimDraft_18_1070375.scaffolds.fasta_scaffold02754_4_8'
 'AntAceMinimDraft_4_1070372.scaffolds.fasta_scaffold18285_3_3'
 'AntAceMinimDraft_18_1070375.scaffolds.fasta_scaffold06919_8_4'
 'AntAceMinimDraft_18_1070375.scaffolds.fasta_scaffold01114_5_1'
 'AntAceMinimDraft_18_1070375.scaffolds.fasta_scaffold01114_5_3'
 'AntAceMinimDraft_18_1070375.scaffolds.fasta_scaffold01114_5_4'
 'AntAceMinimDraft_18_1070375.scaffolds.fasta_scaffold01114_5_5'
 'AntAceMinimDraft_18_1070375.scaffolds.fasta_scaffold01114_5_9'
 'APWor7970453378_1049310.scaffolds.fasta_scaffold03068_1-contig-58_20'
 'APWor7970453378_1049310.scaffolds.fasta_scaffold03068_1-contig-58_21'
 'APWor7970453378_10

In [6]:
godict={}
gocounts = {}

for quid in annotdf.queryid.unique():
    sub = annotdf[annotdf.queryid == quid ]
    gos = ''.join( [terms + ',' for terms in sub.goterms.tolist() ] )
    gos = set(gos.split(','))
    godict[quid] = gos
    for go in gos:
        if go not in gocounts:
            gocounts[go]= 1 
        else:
            gocounts[go] +=1
print(godict)


{'3300014208-Ga0172379-10000243-871512-158_59': set(['', 'GO:0005634', 'GO:0009405', 'GO:0015070', 'GO:0003677']), '3300014208-Ga0172379-10000243-871512-158_58': set(['', 'GO:0003793']), 'AntAceMinimDraft_10_1070366.scaffolds.fasta_scaffold00511_6_2': set(['', 'GO:0009435', 'GO:0004514']), 'AntAceMinimDraft_10_1070366.scaffolds.fasta_scaffold00511_6_3': set(['', 'GO:0016021', 'GO:0007596']), 'AntAceMinimDraft_10_1070366.scaffolds.fasta_scaffold00511_6_4': set(['', 'GO:0006118', 'GO:0016491']), 'AntAceMinimDraft_10_1070366.scaffolds.fasta_scaffold00511_6_5': set(['', 'GO:0004364', 'GO:0004540', 'GO:0006803']), 'AntAceMinimDraft_10_1070366.scaffolds.fasta_scaffold00511_6_8': set(['', 'GO:0005634', 'GO:0003677']), 'AJF63071.1': set(['', 'GO:0009058', 'GO:0016788']), '3300014208-Ga0172379-10000243-871512-158_57': set(['', 'GO:0004945', 'GO:0005554', 'GO:0007186', 'GO:0016021']), 'APWor3302396189_1045246.scaffolds.fasta_scaffold00330_5_19': set(['', 'GO:0005975', 'GO:0008810', 'GO:0006813',

In [7]:
from goatools.base import download_go_basic_obo
obo_fname = download_go_basic_obo()
from goatools.obo_parser import GODag
obodag = GODag("go.obo")

from goatools.associations import read_ncbi_gene2go

geneid2gos = read_ncbi_gene2go("gene2go")
geneid2gos.update(godict)


  EXISTS: go-basic.obo
go.obo: fmt(1.2) rel(2019-02-13) 47,395 GO Terms
  20,397 items READ: gene2go


In [8]:
print(geneid2gos[140])

set(['GO:0001973', 'GO:0005887', 'GO:0008285', 'GO:0008016', 'GO:0009611', 'GO:0032088', 'GO:0001609', 'GO:0007165', 'GO:0006954', 'GO:0005886', 'GO:0007190', 'GO:0007186', 'GO:0030336'])


In [9]:

from goatools.go_enrichment import GOEnrichmentStudy

goeaobj = GOEnrichmentStudy(
        geneid2gos.keys(), # List of mouse protein-coding genes
        geneid2gos, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.1, # default significance cut-off
        methods = ['fdr_bh'])

fisher module not installed.  Falling back on scipy.stats.fisher_exact
100% 20,819 of 20,819 population items found in association


In [10]:
goea_results_all = goeaobj.run_study(godict.keys())


100%    422 of    422 study items found in association
Calculating 18,129 uncorrected p-values using fisher_scipy_stats
  18,129 GO terms are associated with 20,812 of 20,819 population items
     305 GO terms are associated with    415 of    422 study items
     136 GO terms found significant (< 0.1=alpha) after multitest correction: statsmodels fdr_bh


In [14]:
goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.1]

In [15]:
for res in goea_results_sig:
    print(str(res.GO) + '\t' + str(res.p_fdr_bh) )

GO:0006810	1.8234144153067976e-46
GO:0017000	8.357655432810391e-40
GO:0008152	4.2773097381391507e-35
GO:0005975	1.3928274295797841e-21
GO:0006928	4.3952607618452306e-18
GO:0007214	5.989812694756521e-13
GO:0006584	4.900339880630082e-11
GO:0006412	3.443468373658615e-10
GO:0006811	1.9626404136073695e-09
GO:0000752	3.372836498443651e-08
GO:0006099	4.2126583643633007e-08
GO:0006812	6.662325738872567e-07
GO:0006265	7.332850133861393e-07
GO:0009103	1.3175267853425104e-06
GO:0009249	3.419311098001084e-05
GO:0006813	4.5318043253947274e-05
GO:0007600	8.818492721391541e-05
GO:0006308	0.0004017873535177402
GO:0009255	0.002111749139334593
GO:0006310	0.005959219763694984
GO:0015689	0.007174483211440287
GO:0006508	0.007174483211440287
GO:0009058	0.025964824981699556
GO:0009987	0.028088666942239975
GO:0008652	0.052618654911707086
GO:0015979	0.05805766388045323
GO:0006064	0.05805766388045323
GO:0006031	0.05805766388045323
GO:0009252	0.05805766388045323
GO:0009401	0.05805766388045323
GO:0030639	0.058057

In [16]:
for go in gocounts:
    print(go + '\t' + str(gocounts[go]) )

	422
GO:0005267	1
GO:0006887	1
GO:0016218	25
GO:0008803	1
GO:0006807	2
GO:0006470	4
GO:0007596	1
GO:0008964	12
GO:0005788	1
GO:0006779	1
GO:0003797	2
GO:0016916	2
GO:0009165	2
GO:0003794	1
GO:0003793	2
GO:0008270	3
GO:0030115	2
GO:0004799	1
GO:0008374	1
GO:0008375	1
GO:0007001	2
GO:0008643	1
GO:0016597	1
GO:0003918	1
GO:0003713	1
GO:0005549	1
GO:0007517	1
GO:0006304	2
GO:0016998	1
GO:0016624	1
GO:0016021	55
GO:0016020	104
GO:0001584	7
GO:0016003	3
GO:0007214	15
GO:0006308	5
GO:0006265	8
GO:0004563	2
GO:0005529	2
GO:0006260	4
GO:0005177	1
GO:0004890	15
GO:0006268	1
GO:0007010	2
GO:0008248	4
GO:0006865	3
GO:0007608	1
GO:0006281	3
GO:0005093	1
GO:0006014	1
GO:0008076	5
GO:0008246	1
GO:0007602	3
GO:0003767	1
GO:0004945	3
GO:0015098	3
GO:0007155	2
GO:0019904	1
GO:0004146	2
GO:0005496	1
GO:0008474	2
GO:0004949	2
GO:0009073	1
GO:0005000	1
GO:0007160	3
GO:0007283	1
GO:0015485	1
GO:0006823	1
GO:0019538	2
GO:0004635	1
GO:0005554	23
GO:0001619	1
GO:0008880	2
GO:0004412	5
GO:0004556	5
GO:0006371	4